### Target Webscraping

Using Selenium again to go through all the data in Target and see what we can get

Source list
+ https://www.youtube.com/watch?v=SPM1tm2ZdK4&t=855s
+ https://www.youtube.com/watch?v=USrjHgO9Niw python selenium button clicking


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import csv
import re
import pandas as pd 

grocery_list = pd.read_csv('Grocery List.csv')

# college_file_list = ['ames, ia', 'college station, tx'] 
college_file_list = []
with open('CollegeTownsBig.csv', 'r') as file:
    # Create a CSV reader object using DictReader
    for line in file:
        college_file_list.append(line[:-1].replace('-', ', '))

options=Options()
options.add_experimental_option("detach", True)

In [2]:
len(college_file_list)

130

In [3]:
grocery_list = grocery_list.iloc[15:, :]

In [4]:
grocery_df = grocery_list.T
grocery_df.columns = grocery_df.iloc[0]
grocery_df = grocery_df[1:]
grocery_df = grocery_df.reset_index(drop=True)

food_prices = pd.DataFrame(college_file_list, columns=["Location"])
food_prices = pd.concat([food_prices, grocery_df], axis=1)
food_prices = food_prices.iloc[1:, :]
food_prices

,Location,Spaghetti Noodles,Spaghetti Sauce,Green Peppers,Tomato Paste,Onions x2,Apple Juice,Butter 1/4,Coffee 1/4,antihistamine,Ibuprofen
1,"Jacksonville, AL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Livingston, AL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Troy, AL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Fairbanks, AK",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Arkadelphia, AR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
125,"Platteville, WI",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,"River, Falls, WI",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,"Stevens, Point, WI",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,"Whitewater, WI",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                            options=options)

for index, row in grocery_list.iterrows():
    print("Starting on: ", row["Item"])
    
    driver.get(row["URL"])
    time.sleep(3)
    for college in college_file_list:

        # Find the button by its ID
        webStoreButton = driver.find_element(By.ID, "web-store-id-msg-btn")

        # Click the button
        webStoreButton.click()
        time.sleep(4)

        inputBox = driver.find_element(By.ID, "zip-or-city-state")
        inputBox.send_keys(college) # Need to input "city, st"

        lookupButton =  driver.find_element(By.CSS_SELECTOR, '[data-test="@web/StoreLocationSearch/button"]')
        lookupButton.click()
        time.sleep(4)

        ShopStoreButton =  driver.find_element(By.CSS_SELECTOR, '[data-test="@web/StoreMenu/ShopThisStoreButton"]')
        ShopStoreButton.click()

        time.sleep(3)
        
        if re.search('data-test="product-price"', driver.page_source):
            # Putting the value in last so that we can ensure on the first and last iteration it will be in the right spot
            price_element = driver.find_element(By.CSS_SELECTOR, '[data-test="product-price"]')
            food_prices.loc[food_prices['Location'] == college, row['Item']] = price_element.text
        else:
            print("Error value at: ", college, " for item: ", row["Item"])
        time.sleep(3)


driver.close()

Starting on:  Spaghetti Noodles
Starting on:  Spaghetti Sauce
Starting on:  Green Peppers
Starting on:  Tomato Paste
Starting on:  Onions x2
Starting on:  Apple Juice
Starting on:  Butter 1/4
Starting on:  Coffee 1/4
Starting on:  antihistamine
Starting on:  Ibuprofen


In [6]:
columns_to_convert = food_prices.columns
columns_to_convert = columns_to_convert[1:15]
columns_to_convert

food_prices2 = food_prices.copy()
food_prices2[columns_to_convert] = food_prices2[columns_to_convert].applymap(lambda x: pd.to_numeric(x.replace('$', '')))

row_sum = food_prices2[columns_to_convert].sum(axis=1)
food_prices2['Row Sum'] = round(row_sum,2)
food_prices2

,Location,Spaghetti Noodles,Spaghetti Sauce,Green Peppers,Tomato Paste,Onions x2,Apple Juice,Butter 1/4,Coffee 1/4,antihistamine,Ibuprofen,Row Sum
1,"Jacksonville, AL",1.89,1.69,1.89,0.85,0.99,1.99,5.79,15.49,7.99,3.99,42.56
2,"Livingston, AL",1.89,1.69,1.89,0.85,0.99,1.99,5.79,15.49,7.99,3.99,42.56
3,"Troy, AL",1.89,1.69,1.89,0.85,0.99,1.99,5.79,15.49,7.99,3.99,42.56
4,"Fairbanks, AK",1.89,1.69,1.89,0.85,0.99,1.99,5.79,13.29,7.99,3.99,40.36
5,"Arkadelphia, AR",1.89,1.69,1.89,0.85,0.95,1.99,5.79,13.29,7.99,3.99,40.32
...,...,...,...,...,...,...,...,...,...,...,...,...
125,"Platteville, WI",1.89,1.69,1.89,0.85,1.19,1.99,5.79,13.29,7.99,3.99,40.56
126,"River, Falls, WI",1.89,1.69,1.89,0.85,1.19,1.99,5.79,13.29,7.99,3.99,40.56
127,"Stevens, Point, WI",1.89,1.69,1.89,0.85,1.19,1.99,5.79,13.29,7.99,3.99,40.56
128,"Whitewater, WI",1.89,1.69,1.89,0.85,1.19,1.99,5.79,13.29,7.99,3.99,40.56


In [7]:
food_prices2.to_csv("College Food Prices Big Half 2.csv", index=False)